<a href="https://colab.research.google.com/github/ANUSHA474-ai/Fashion-Tech-AI-ML/blob/main/Fashion_Tech_AI_ML_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import warnings
import os

# Suppress warnings and progress bars
warnings.filterwarnings('ignore')
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

print("=" * 80)
print("FASHION TECH AI/ML CANDIDATE SHORTLISTING MODEL")
print("Using Text Similarity Analysis")
print("=" * 80)


# 1. JOB DESCRIPTION


job_description = """
Fashion Tech AI/ML Engineer

We are seeking an experienced AI/ML Engineer to join our Fashion Tech team.
The ideal candidate will have strong expertise in computer vision, deep learning,
and recommendation systems applied to the fashion industry.

Key Responsibilities:
- Develop and deploy computer vision models for fashion item recognition
- Build recommendation systems for personalized styling
- Work with large-scale fashion datasets and image processing
- Implement deep learning models using PyTorch or TensorFlow
- Collaborate with fashion designers and product teams
- Deploy ML models in production environments

Required Skills:
- 3+ years of experience in machine learning and deep learning
- Strong proficiency in Python, PyTorch, and TensorFlow
- Experience with computer vision (CNN, image classification, object detection)
- Knowledge of NLP and recommendation systems
- Experience with fashion domain or e-commerce preferred
- Strong problem-solving and communication skills
- Master's degree in Computer Science, AI, or related field

Bonus Skills:
- Experience with GANs for fashion generation
- Knowledge of fashion trends and styling
- Experience with cloud platforms (AWS, GCP, Azure)
- Published research in ML/CV conferences
"""

print("\n📋 JOB DESCRIPTION:")
print("-" * 80)
print(job_description[:500] + "...\n")

# 2. CANDIDATE PROFILES

candidates = {
    "Candidate_1": """
    Senior ML Engineer with 5 years of experience in computer vision and deep learning.
    Expert in PyTorch and TensorFlow. Built CNN models for image classification and
    object detection. Worked on fashion e-commerce recommendation systems using
    collaborative filtering. Experience with GANs for image generation. Published
    2 papers at CVPR. Strong Python skills and deployed models on AWS. Master's in
    Computer Science from Stanford. Passionate about fashion tech and AI.
    """,

    "Candidate_2": """
    AI Research Scientist with 4 years in NLP and recommendation systems.
    Proficient in Python, scikit-learn, and TensorFlow. Built recommendation
    engines for e-commerce platforms. Experience with transformers and BERT.
    PhD in Machine Learning from MIT. Strong publication record. Limited
    computer vision experience but quick learner. Interested in fashion applications.
    """,

    "Candidate_3": """
    Junior ML Engineer with 2 years experience. Good knowledge of Python and
    basic machine learning algorithms. Completed online courses in deep learning.
    Built small projects with TensorFlow. No production deployment experience.
    Bachelor's degree in Computer Engineering. Enthusiastic about learning
    computer vision and fashion tech.
    """,

    "Candidate_4": """
    Full Stack Developer with 6 years experience in web development.
    Proficient in JavaScript, React, Node.js. Some experience with Python
    and basic ML libraries. Built web applications for fashion brands.
    Strong frontend skills but limited ML knowledge. Bachelor's in
    Information Technology. Willing to transition into ML engineering.
    """,

    "Candidate_5": """
    Computer Vision Engineer with 4 years specializing in fashion tech.
    Expert in CNN, image segmentation, and style transfer. Built virtual
    try-on systems using deep learning. Proficient in PyTorch, OpenCV, and
    Python. Experience with fashion datasets like DeepFashion. Worked at
    fashion e-commerce startup. Master's in AI. Strong understanding of
    fashion trends and visual aesthetics.
    """,

    "Candidate_6": """
    Data Scientist with 3 years in recommendation systems and personalization.
    Experience with collaborative filtering, matrix factorization, and deep
    learning recommenders. Proficient in Python, TensorFlow, and SQL. Built
    product recommendation systems for retail. Some computer vision experience
    with image embeddings. Master's in Data Science. Excited about fashion AI.
    """
}

candidate_names = list(candidates.keys())
print(f"📊 Total Candidates: {len(candidates)}\n")

# 3. TEXT PREPROCESSING

def preprocess_text(text):
    """Clean and normalize text"""
    text = text.lower().strip()
    text = ' '.join(text.split())
    return text

job_desc_processed = preprocess_text(job_description)
candidates_processed = {name: preprocess_text(profile)
                       for name, profile in candidates.items()}

print("✅ Text preprocessing completed\n")


# 4. METHOD 1: TF-IDF + COSINE SIMILARITY

print("=" * 80)
print("METHOD 1: TF-IDF + COSINE SIMILARITY")
print("=" * 80)

# Create corpus
corpus = [job_desc_processed] + list(candidates_processed.values())

# Initialize and fit TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=500,
    ngram_range=(1, 2),
    stop_words='english'
)

tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# Calculate similarity
job_vector = tfidf_matrix[0:1]
candidate_vectors = tfidf_matrix[1:]
tfidf_similarities = cosine_similarity(job_vector, candidate_vectors)[0]

# Results
tfidf_results = pd.DataFrame({
    'Candidate': candidate_names,
    'TF-IDF Similarity': tfidf_similarities,
    'TF-IDF Score (%)': (tfidf_similarities * 100).round(2)
}).sort_values('TF-IDF Similarity', ascending=False)

print("\n📊 TF-IDF Similarity Scores:")
print("-" * 80)
print(tfidf_results.to_string(index=False))
print()

# 5. METHOD 2: BERT EMBEDDINGS + COSINE SIMILARITY

print("\n" + "=" * 80)
print("METHOD 2: BERT EMBEDDINGS + COSINE SIMILARITY")
print("=" * 80)

# Load BERT model (suppressing download progress)
print("\n🤖 Loading Sentence-BERT model (all-MiniLM-L6-v2)...")
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')
print("✅ Model loaded successfully\n")

# Generate embeddings
print("🔄 Generating semantic embeddings...")
job_embedding = model.encode([job_desc_processed])[0]
candidate_embeddings = model.encode(list(candidates_processed.values()))
print("✅ Embeddings generated\n")

# Calculate similarity
bert_similarities = cosine_similarity(
    job_embedding.reshape(1, -1),
    candidate_embeddings
)[0]

# Results
bert_results = pd.DataFrame({
    'Candidate': candidate_names,
    'BERT Similarity': bert_similarities,
    'BERT Score (%)': (bert_similarities * 100).round(2)
}).sort_values('BERT Similarity', ascending=False)

print("📊 BERT Similarity Scores:")
print("-" * 80)
print(bert_results.to_string(index=False))
print()

# 6. COMBINED SCORING & FINAL RANKING

print("\n" + "=" * 80)
print("COMBINED SCORING & FINAL RANKING")
print("=" * 80)

# Merge results
final_results = pd.merge(
    tfidf_results[['Candidate', 'TF-IDF Similarity']],
    bert_results[['Candidate', 'BERT Similarity']],
    on='Candidate'
)

# Combined score: 40% TF-IDF + 60% BERT
final_results['Combined Score'] = (
    0.4 * final_results['TF-IDF Similarity'] +
    0.6 * final_results['BERT Similarity']
)
final_results['Combined Score (%)'] = (final_results['Combined Score'] * 100).round(2)

# Sort by combined score
final_results = final_results.sort_values('Combined Score', ascending=False)

print("\n📊 Final Combined Scores:")
print("-" * 80)
print(final_results.to_string(index=False))
print()

# 7. SHORTLISTING DECISIONS

print("\n" + "=" * 80)
print("SHORTLISTING DECISIONS")
print("=" * 80)

# Define thresholds
EXCELLENT_THRESHOLD = 0.55
GOOD_THRESHOLD = 0.45
SHORTLIST_THRESHOLD = 0.40

def get_decision(score):
    if score >= EXCELLENT_THRESHOLD:
        return "✅ STRONG SHORTLIST"
    elif score >= GOOD_THRESHOLD:
        return "✓ SHORTLIST"
    elif score >= SHORTLIST_THRESHOLD:
        return "⚠ MAYBE"
    else:
        return "❌ REJECT"

def get_reasoning(row):
    """Generate explanation for decision"""
    reasons = []

    if row['Combined Score'] >= 0.55:
        reasons.append("Excellent match with job requirements")
    if row['TF-IDF Similarity'] >= 0.30:
        reasons.append("Strong technical keyword alignment")
    if row['BERT Similarity'] >= 0.60:
        reasons.append("High semantic similarity to role")
    if row['TF-IDF Similarity'] < 0.20:
        reasons.append("Limited keyword matching")
    if row['BERT Similarity'] < 0.40:
        reasons.append("Low semantic alignment")

    return " | ".join(reasons) if reasons else "Standard evaluation"

final_results['Decision'] = final_results['Combined Score'].apply(get_decision)
final_results['Reasoning'] = final_results.apply(get_reasoning, axis=1)

print("\n🎯 Shortlisting Recommendations:")
print("-" * 80)

for idx, row in final_results.iterrows():
    print(f"\n{row['Candidate']}:")
    print(f"  Combined Score: {row['Combined Score (%)']:.2f}%")
    print(f"  TF-IDF: {row['TF-IDF Similarity']:.3f} | BERT: {row['BERT Similarity']:.3f}")
    print(f"  Decision: {row['Decision']}")
    print(f"  Reasoning: {row['Reasoning']}")

# 8. SUMMARY & STATISTICS

print("\n\n" + "=" * 80)
print("SUMMARY & STATISTICS")
print("=" * 80)

decision_counts = final_results['Decision'].value_counts()

print("\n📊 Decision Breakdown:")
print("-" * 80)
for decision in ["✅ STRONG SHORTLIST", "✓ SHORTLIST", "⚠ MAYBE", "❌ REJECT"]:
    count = decision_counts.get(decision, 0)
    percentage = (count / len(final_results)) * 100
    print(f"{decision}: {count} candidates ({percentage:.1f}%)")

print("\n🏆 Top 3 Recommended Candidates:")
print("-" * 80)
top_3 = final_results.head(3)
for i, (idx, row) in enumerate(top_3.iterrows(), 1):
    print(f"{i}. {row['Candidate']} - {row['Combined Score (%)']:.2f}% match")

print("\n📈 Score Statistics:")
print("-" * 80)
print(f"Mean Score: {final_results['Combined Score'].mean():.3f} ({final_results['Combined Score'].mean()*100:.1f}%)")
print(f"Median Score: {final_results['Combined Score'].median():.3f} ({final_results['Combined Score'].median()*100:.1f}%)")
print(f"Std Dev: {final_results['Combined Score'].std():.3f}")
print(f"Range: {final_results['Combined Score'].min():.3f} - {final_results['Combined Score'].max():.3f}")